In [1]:
import pandas as pd
import numpy as np
trips = pd.read_csv("psrc_trips.csv")
trips['weight'] = np.sum(trips[['trip_adult_weight_2021', 'trip_respondent_weight_2021',
       'trip_weight_2017', 'trip_weight_2019', 'trip_weight_2017_2019',
       'trip_adult_weight_2017_2019']].fillna(0).values,axis=1)


<ipython-input-1-d9ac6fb59f80>:3: DtypeWarning: Columns (5,8,25,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,52,68,69,73,75,76,79,86,87,88,89,90,91,92,93,94,95,96,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,130,131,132,133,134,135,136,137,138,139,140,142,154) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("psrc_trips.csv")
<ipython-input-1-d9ac6fb59f80>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  x = data.groupby("person_dim_id")["vehicle_count","hhincome_broad","car_share","numadults","numchildren","age_category","gender","employment","education","license"].first().reset_index()


In [14]:
trips["depart_time"] = pd.to_datetime(trips["depart_time_string"].str.replace("Date: ","")).dt.hour
trips["depart_time"]
trips["21-5c"] = ((trips["depart_time"]>=21) | (trips["depart_time"]<5)) & (trips['o_tract10']!=trips["d_tract10"])
trips["5-13c"] = ((trips["depart_time"]>=5) & (trips["depart_time"]<13)) & (trips['o_tract10']!=trips["d_tract10"])
trips["13-21c"] = ((trips["depart_time"]>=13) & (trips["depart_time"]<21)) & (trips['o_tract10']!=trips["d_tract10"])
trips["21-5s"] = ((trips["depart_time"]>=21) | (trips["depart_time"]<5)) & (trips['o_tract10']==trips["d_tract10"])
trips["5-13s"] = ((trips["depart_time"]>=5) & (trips["depart_time"]<13)) & (trips['o_tract10']==trips["d_tract10"])
trips["13-21s"] = ((trips["depart_time"]>=13) & (trips["depart_time"]<21)) & (trips['o_tract10']==trips["d_tract10"])

In [17]:
data = trips[["person_dim_id","vehicle_count","hhincome_broad","car_share","numadults","numchildren","age_category","gender","employment","education","license","o_tract10", "21-5c","5-13c", "13-21c", "21-5s", "5-13s", "13-21s",'weight']].dropna(subset=['o_tract10'])
data = data.fillna('Unknown').replace("Missing: Skip Logic", "Unknown").replace("Prefer not to answer", "Unknown").replace("Not listed here / prefer not to answer", "Unknown")
x = data.groupby("person_dim_id")["vehicle_count","hhincome_broad","car_share","numadults","numchildren","age_category","gender","employment","education","license","o_tract10"].first().reset_index()
w = data.groupby("person_dim_id")["weight"].first().reset_index()
y = data.groupby("person_dim_id")["21-5c","5-13c", "13-21c", "21-5s", "5-13s", "13-21s" ].sum().reset_index()
x.to_csv("x.csv")
y.to_csv("y.csv")
w.to_csv("w.csv")

<ipython-input-17-b279ecacfd2b>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  x = data.groupby("person_dim_id")["vehicle_count","hhincome_broad","car_share","numadults","numchildren","age_category","gender","employment","education","license","o_tract10"].first().reset_index()
<ipython-input-17-b279ecacfd2b>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  y = data.groupby("person_dim_id")["21-5c","5-13c", "13-21c", "21-5s", "5-13s", "13-21s" ].sum().reset_index()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

class FirstModel():
    def __init__(self):
        self.pls = PLSRegression(n_components=10)
        self.encoder = OneHotEncoder()
        self.linear = LinearRegression()
        self.rf = RandomForestRegressor()
        self.knn = KNeighborsRegressor()

    def train(self, x, y, w):
      x,y,w = x.iloc[:,1:], y.iloc[:,1:].values, w.iloc[:,1:].values
      x = self.encoder.fit_transform(x.values).toarray()
      x, _ = self.pls.fit_transform(x, y)
      self.linear.fit(x,y,sample_weight = w.ravel())
      self.rf.fit(x,y,sample_weight = w.ravel())

    def predict(self, features, mode):
      x = self.pls.transform(self.encoder.transform([features]).toarray())
      if mode == "Random Forest":
        return self.rf.predict(x)
      else:
        return self.linear.predict(x)

model = FirstModel()
model.train(x,y,w)
model.predict(x.iloc[1,1:].values.tolist(),"Random Forest")

<ipython-input-3-9bd98988efc2>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.rf.fit(x,y,sample_weight = w.ravel())


array([3.38358354])

In [ ]:
from joblib import dump
dump(model, 'first_model.joblib')

['first_model.joblib']

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler object
scaler = StandardScaler()

first_phase = pd.read_csv("first_phase.csv", index_col=0)
cluster = pd.read_csv("spectral_clustering_result.csv", index_col=0)
cluster['final_home_tract10'] = cluster['Census Tract']
data = first_phase.merge(cluster, on="final_home_tract10")
x_variable = ['vehicle_count',
       'hhincome_broad', 'car_share', 'numadults', 'numchildren',
       'age_category', 'gender', 'employment', 'education', 'license']
x = pd.DataFrame(scaler.fit_transform(data[x_variable]), columns=x_variable)
y = data[['trip_id']]
weight = data[['weight','Cluster']]

In [ ]:
sample_weights_list = []
aug_factor = 10
for value in np.unique(weight['Cluster'].values):
  result = weight.apply(lambda row: row['weight'] * 10 if row['Cluster'] == value else row['weight'], axis=1)
  sample_weights_list.append(result)

In [ ]:
class WeightedRegressor:
    def __init__(self, regressor):
        self.regressor = regressor
        self.models = []

    def fit(self, X, y, sample_weights_list):
        self.models = []
        for weights in sample_weights_list:
            model = self.regressor.__class__()  # Create a new instance of the regressor
            model.fit(X, y, sample_weight=weights)
            self.models.append(model)
        return self

    def predict(self, X):
        y_preds = [model.predict(X) for model in self.models]
        return y_preds

# Example usage:
from sklearn.ensemble import RandomForestRegressor



# Initialize WeightedRegressor with a RandomForestRegressor
weighted_rf_regressor = WeightedRegressor(RandomForestRegressor())

# Train the models with different weightings
weighted_rf_regressor.fit(x, y, sample_weights_list)


<ipython-input-3-8db40055f3df>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y, sample_weight=weights)
<ipython-input-3-8db40055f3df>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y, sample_weight=weights)
<ipython-input-3-8db40055f3df>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y, sample_weight=weights)
<ipython-input-3-8db40055f3df>:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X, y, sample_weight=weights)
<ipython-input-3-8db40055f3df>:10: DataConversionWarning: A column-vector y 

In [ ]:
!pip install shap
import numpy as np
import shap

def compute_feature_importance(models, X_train, method='shap'):
    feature_importance = {}
    for i, model in enumerate(models):
        if method == 'shap':
            explainer = shap.Explainer(model, X_train)
            shap_values = explainer.shap_values(X_train)
            feature_importance[f'Model_{i+1}'] = np.abs(np.mean(shap_values, axis=0))
    return feature_importance

# Compute feature importance across all models
all_feature_importance = compute_feature_importance(weighted_rf_regressor.models, x, method='shap')

# Aggregate feature importance across all models
aggregate_feature_importance = {}
for model_feature_importance in all_feature_importance.values():
    for feature, importance in enumerate(model_feature_importance):
        if feature not in aggregate_feature_importance:
            aggregate_feature_importance[feature] = importance
        else:
            aggregate_feature_importance[feature] += importance

# Normalize aggregated feature importance
total_sum = sum(aggregate_feature_importance.values())
normalized_feature_importance = {feature: importance / total_sum for feature, importance in aggregate_feature_importance.items()}

# Sort by importance
sorted_feature_importance = dict(sorted(normalized_feature_importance.items(), key=lambda item: item[1], reverse=True))

# Print or visualize sorted feature importance
print("Feature Importance:")
for feature, importance in sorted_feature_importance.items():
    print(f"Feature {feature+1}: Importance = {importance:.4f}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.7/535.7 kB 8.2 MB/s eta 0:00:00


 64%|=============       | 6585/10267 [08:47<04:54]       